Import librerie

In [1]:
import libProject # libreria del progetto

In [2]:
import torch
import numpy as np
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, classification_report
from os.path import join
import time
import pandas as pd
import ast
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
import ssl
import json
ssl._create_default_https_context = ssl._create_unverified_context

Lettura CSV

In [3]:
fd = pd.read_csv('../Dataset/TrackFeatures4.csv')

Preprocessing e creazione Training e Test set

In [4]:
X_scaled, y_encoded = libProject.doPreprocessing(fd)

# Salva X_scaled e y_encoded in un file JSON
with open('preprocessed_data.json', 'w') as f_json:
    json.dump({
        'X_scaled': X_scaled.tolist(),
        'y_encoded': y_encoded.tolist()
    }, f_json)

# Carica X_scaled e y_encoded dal file JSON
with open('preprocessed_data.json', 'r') as f_json:
    data = json.load(f_json)
    X_scaled = np.array(data['X_scaled'])
    y_encoded = np.array(data['y_encoded'])

#distribuzione in train e test
x_train, x_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=123, stratify=y_encoded
)

# Crea i dataset
train_dataset = libProject.AudioFeaturesDataset(x_train, y_train)
test_dataset   = libProject.AudioFeaturesDataset(x_test, y_test)

# Crea i DataLoader
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

Preprocessing in corso...

Preprocessing riga 0...

Preprocessing riga 1...

Preprocessing riga 2...

Preprocessing riga 3...

Preprocessing riga 4...

Preprocessing riga 5...

Preprocessing riga 6...

Preprocessing riga 7...

Preprocessing riga 8...

Preprocessing riga 9...

Preprocessing riga 10...

Preprocessing riga 11...

Preprocessing riga 12...

Preprocessing riga 13...

Preprocessing riga 14...

Preprocessing riga 15...

Preprocessing riga 16...

Preprocessing riga 17...

Preprocessing riga 18...

Preprocessing riga 19...

Preprocessing riga 20...

Preprocessing riga 21...

Preprocessing riga 22...

Preprocessing riga 23...

Preprocessing riga 24...

Preprocessing riga 25...

Preprocessing riga 26...

Preprocessing riga 27...

Preprocessing riga 28...

Preprocessing riga 29...

Preprocessing riga 30...

Preprocessing riga 31...

Preprocessing riga 32...

Preprocessing riga 33...

Preprocessing riga 34...

Preprocessing riga 35...

Preprocessing riga 36...

Preprocessing riga 37

In [5]:
print("Input size:", x_train.shape[1])
print("Output size:", len(np.unique(y_train)))

Input size: 5944
Output size: 29


In [6]:
name = 'LogisticRegressor'
lr = 0.1
epochs = 1000
momentum = 0.99
model = libProject.LogisticRegressor(5944, 29)
model = libProject.train_classifier(model, train_loader, test_loader, exp_name=f"{name}_{int(time.time())}", lr=lr, epochs=epochs, momentum=momentum)

predictions_train, labels_train = libProject.test_classifier(model, train_loader)
predictions_test, labels_test = libProject.test_classifier(model, test_loader)

with open('metrics.json', 'r') as f:
    metrics = json.load(f)

metrics.append({
    'model': 'LogisticRegressor',
    'parameters': {
        'learning_rate': lr,
        'epochs': epochs,
        'momentum': momentum
    },
    'train_classification_report': classification_report(labels_train, predictions_train, output_dict=True),
    'test_classification_report': classification_report(labels_test, predictions_test, output_dict=True),
})

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

Epoch 1/1000
Epoch 2/1000
Epoch 3/1000
Epoch 4/1000
Epoch 5/1000
Epoch 6/1000
Epoch 7/1000
Epoch 8/1000
Epoch 9/1000
Epoch 10/1000
Epoch 11/1000
Epoch 12/1000
Epoch 13/1000
Epoch 14/1000
Epoch 15/1000
Epoch 16/1000
Epoch 17/1000
Epoch 18/1000
Epoch 19/1000
Epoch 20/1000
Epoch 21/1000
Epoch 22/1000
Epoch 23/1000
Epoch 24/1000
Epoch 25/1000
Epoch 26/1000
Epoch 27/1000
Epoch 28/1000
Epoch 29/1000
Epoch 30/1000
Epoch 31/1000
Epoch 32/1000
Epoch 33/1000
Epoch 34/1000
Epoch 35/1000
Epoch 36/1000
Epoch 37/1000
Epoch 38/1000
Epoch 39/1000
Epoch 40/1000
Epoch 41/1000
Epoch 42/1000
Epoch 43/1000
Epoch 44/1000
Epoch 45/1000
Epoch 46/1000
Epoch 47/1000
Epoch 48/1000
Epoch 49/1000
Epoch 50/1000
Epoch 51/1000
Epoch 52/1000
Epoch 53/1000
Epoch 54/1000
Epoch 55/1000
Epoch 56/1000
Epoch 57/1000
Epoch 58/1000
Epoch 59/1000
Epoch 60/1000
Epoch 61/1000
Epoch 62/1000
Epoch 63/1000
Epoch 64/1000
Epoch 65/1000
Epoch 66/1000
Epoch 67/1000
Epoch 68/1000
Epoch 69/1000
Epoch 70/1000
Epoch 71/1000
Epoch 72/1000
E

In [7]:
libProject.print_metrics_from_json('metrics.json')


Model: LogisticRegressor
  learning_rate: 0.01
  epochs: 150
  momentum: 0.99

  train_classification_report:
    accuracy: 0.12493786246893124
    macro avg:
      precision: 0.2520694783425895
      recall: 0.11938085923889222
      f1-score: 0.06157549594788728
      support: 6035.0
    weighted avg:
      precision: 0.24038302122775407
      recall: 0.12493786246893124
      f1-score: 0.06821384734908384
      support: 6035.0

  test_classification_report:
    accuracy: 0.10669317428760769
    macro avg:
      precision: 0.04943418244155853
      recall: 0.10234407242520834
      f1-score: 0.03835876528602472
      support: 1509.0
    weighted avg:
      precision: 0.05881240252608794
      recall: 0.10669317428760769
      f1-score: 0.04426206536464249
      support: 1509.0

Model: LogisticRegressor
  learning_rate: 0.1
  epochs: 150
  momentum: 0.99

  train_classification_report:
    accuracy: 0.12410936205468102
    macro avg:
      precision: 0.3051539478676257
      recall: 